# Training a Classifier on the *Salammbô* Dataset with PyTorch
Author: Pierre Nugues

We first import some modules

In [26]:
import torch
import torch.nn as nn
import random

We set a seed to have reproducible results

In [27]:
random.seed(4321)
torch.manual_seed(4321)

### Reading the dataset
We can read the data from a file with the svmlight format or directly create numpy arrays

In [28]:
X = torch.Tensor(
    [[35680, 2217], [42514, 2761], [15162, 990], [35298, 2274],
     [29800, 1865], [40255, 2606], [74532, 4805], [37464, 2396],
     [31030, 1993], [24843, 1627], [36172, 2375], [39552, 2560],
     [72545, 4597], [75352, 4871], [18031, 1119], [36961, 2503],
     [43621, 2992], [15694, 1042], [36231, 2487], [29945, 2014],
     [40588, 2805], [75255, 5062], [37709, 2643], [30899, 2126],
     [25486, 1784], [37497, 2641], [40398, 2766], [74105, 5047],
     [76725, 5312], [18317, 1215]
     ])

y = torch.Tensor(
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Scaling the Data
Scaling and normalizing are usually very significant with neural networks. We use sklean transformers. They consist of two main methods: `fit()` and `transform()`.

### Normalizing

In [29]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
X_norm = normalizer.fit_transform(X)
X_norm[:4]

array([[0.9980751 , 0.06201605],
       [0.99789774, 0.06480679],
       [0.9978751 , 0.06515607],
       [0.9979313 , 0.06428964]], dtype=float32)

### Standardizing

In [30]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=True, with_std=True)
X_scaled = scaler.fit_transform(X_norm)
X_scaled[:4]

array([[ 1.6832309 , -1.7197776 ],
       [ 0.5732034 , -0.5614551 ],
       [ 0.431466  , -0.41648194],
       [ 0.7831985 , -0.7761007 ]], dtype=float32)

## Creating PyTorch Tensors
PyTorch has its own implementation of matrices called tensors. They are more or less equivalent to NumPy arrays. We need to convert our dataset to these tensors and represent $\mathbf{y}$  as a column vector

In [31]:
Y = y.reshape((-1, 1))
Y[:4]

tensor([[0.],
        [0.],
        [0.],
        [0.]])

In [32]:
X_scaled = torch.Tensor(X_scaled)
X_scaled[:4]

tensor([[ 1.6832, -1.7198],
        [ 0.5732, -0.5615],
        [ 0.4315, -0.4165],
        [ 0.7832, -0.7761]])

## Creating a Model

We define a classifier equivalent to a logistic regression

In [33]:
input_dim = X.shape[1]

In [34]:
model = nn.Sequential(
    torch.nn.Linear(input_dim, 1),
    torch.nn.Sigmoid())
model

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
  (1): Sigmoid()
)

We create the model. To try the network with one hidden layer, set `complex` to true

In [35]:
model[0].weight, model[0].bias

(Parameter containing:
 tensor([[-0.5296,  0.0533]], requires_grad=True),
 Parameter containing:
 tensor([0.2212], requires_grad=True))

In [36]:
loss_fn = nn.BCELoss()    # binary cross entropy loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

## Fitting the Model
We will show three methods: batch gradient descent, stochastic descent, and minibatches

### Batch Gradient Descent

We fit the whole dataset (batch gradient descent)

In [37]:
model.train()               # sets PyTorch in the train mode
for epoch in range(100):
    Y_pred = model(X_scaled)
    loss = loss_fn(Y_pred, Y)
    optimizer.zero_grad()   # resets the gradients
    loss.backward()         # gradient backpropagation
    optimizer.step()        # weight updates

### Stochastic Gradient Descent

or, we fit the model with a batch size of one item (stochastic gradient descent)

In [38]:
model.train()
for epoch in range(50):
    for x_scaled, y in zip(X_scaled, Y):
        y_pred = model(x_scaled)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

### Minibatch Gradient Descent

Or we fit it with mini-batches, first with a simple inner loop

In [39]:
batch_size = 4
model.train()
for epoch in range(50):
    # Would need to shuffle X and y
    for i in range(0, X_scaled.size()[0], batch_size):
        Y_batch_pred = model(X_scaled[i:i + batch_size])
        loss = loss_fn(Y_batch_pred, Y[i:i + batch_size])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Then with a dataloader

In [40]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X_scaled, Y)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [41]:
model.train()
for epoch in range(50):
    for X_scaled_batch, Y_batch in dataloader:
        Y_batch_pred = model(X_scaled_batch)
        loss = loss_fn(Y_batch_pred, Y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

## The weights

In [42]:
list(model.parameters())

[Parameter containing:
 tensor([[-2.2376,  1.7657]], requires_grad=True),
 Parameter containing:
 tensor([0.2118], requires_grad=True)]

Also in the form of a dictionary

In [43]:
model.state_dict()

OrderedDict([('0.weight', tensor([[-2.2376,  1.7657]])),
             ('0.bias', tensor([0.2118]))])

## Prediction

### Probabilities

We compute the probabilities to belong to class 1 for all the training set

In [44]:
model.eval()
y_pred_proba = model(X_scaled)
y_pred_proba

tensor([[0.0014],
        [0.1128],
        [0.1841],
        [0.0516],
        [0.0028],
        [0.0832],
        [0.0554],
        [0.0248],
        [0.0382],
        [0.2375],
        [0.2904],
        [0.0817],
        [0.0098],
        [0.0722],
        [0.0012],
        [0.9256],
        [0.9816],
        [0.5792],
        [0.9832],
        [0.8517],
        [0.9922],
        [0.8534],
        [0.9985],
        [0.9871],
        [0.9983],
        [0.9992],
        [0.9776],
        [0.9595],
        [0.9937],
        [0.5536]], grad_fn=<SigmoidBackward0>)

We recompute it with matrices

In [45]:
m_params = list(model.parameters())

In [50]:
torch.sigmoid(X_scaled @ m_params[0].T + m_params[1])[:4]

tensor([[0.0014],
        [0.1128],
        [0.1841],
        [0.0516]], grad_fn=<SliceBackward0>)

### Classes

In [47]:
def predict_class(y_pred_proba):
    y_pred = torch.zeros(y_pred_proba.shape[0])
    for i in range(y_pred_proba.shape[0]):
        if y_pred_proba[i][0] >= 0.5:
            y_pred[i] = 1
    return y_pred

In [48]:
y_pred = predict_class(y_pred_proba)
y_pred

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

## Evaluation

In [49]:
from sklearn.metrics import classification_report

print(classification_report(Y, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        15
         1.0       1.00      1.00      1.00        15

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



We computed the accuracy from the training set. This is not a good practice. We should use a dedicated test set instead.